# Capstone
### This project will be used for coursera Capstone course 

In [101]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.utils import resample
from sklearn import preprocessing

In [102]:
df_original = pd.read_csv('Data-Collisions.csv')
df = df_original.copy()
df_original.info()

/Users/SvetlanaKorchagina/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (33) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 194673 entries, 0 to 194672
Data columns (total 38 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   SEVERITYCODE    194673 non-null  int64  
 1   X               189339 non-null  float64
 2   Y               189339 non-null  float64
 3   OBJECTID        194673 non-null  int64  
 4   INCKEY          194673 non-null  int64  
 5   COLDETKEY       194673 non-null  int64  
 6   REPORTNO        194673 non-null  object 
 7   STATUS          194673 non-null  object 
 8   ADDRTYPE        192747 non-null  object 
 9   INTKEY          65070 non-null   float64
 10  LOCATION        191996 non-null  object 
 11  EXCEPTRSNCODE   84811 non-null   object 
 12  EXCEPTRSNDESC   5638 non-null    object 
 13  SEVERITYCODE.1  194673 non-null  int64  
 14  SEVERITYDESC    194673 non-null  object 
 15  COLLISIONTYPE   189769 non-null  object 
 16  PERSONCOUNT     194673 non-null  int64  
 17  PEDCOUNT  

# Preprocessing

In [103]:
columns_to_drop = ['X', 'Y','SDOTCOLNUM', 'ST_COLDESC', 'SDOT_COLDESC', 'COLDETKEY', 'INCKEY', 'OBJECTID', 'INTKEY', 'LOCATION', 'EXCEPTRSNCODE', 'EXCEPTRSNDESC', 'INCDATE', 'INATTENTIONIND', 'PEDROWNOTGRNT', 'SPEEDING']

In [104]:
# unnessesary columns dropped
df.drop(columns = columns_to_drop, inplace = True)

In [105]:
# coverted column with date and time of accident to datetime format
df['INCDTTM'] = pd.to_datetime(df['INCDTTM'])

In [106]:
# dropping unknown
df.drop(index = df[df['ST_COLCODE'] == ' '].index, inplace = True)
df.dropna(axis = 0, inplace = True)

In [107]:
# converting data in columns from str to numeric
df['UNDERINFL'].replace(to_replace=['N','Y'], value=[0,1],inplace=True)
df['UNDERINFL'] = pd.to_numeric(df['UNDERINFL'])

df['HITPARKEDCAR'].replace(to_replace=['N','Y'], value=[0,1],inplace=True)
df['STATUS'].replace(to_replace=['Unmatched','Matched'], value=[0,1],inplace=True)


df['ST_COLCODE'] = pd.to_numeric(df['ST_COLCODE'])

In [108]:
# filling nan values with Unknown 
df['WEATHER'].fillna('Unknown', inplace = True)
df['ROADCOND'].fillna('Unknown', inplace = True)
df['LIGHTCOND'].fillna('Unknown', inplace = True)
df['JUNCTIONTYPE'].fillna('Unknown', inplace = True)
df['COLLISIONTYPE'].fillna('Unknown', inplace = True)

In [109]:
# dropping duplicates
df.drop_duplicates(subset = 'REPORTNO', keep = 'first', inplace=True)

# Down-sample Majority Class

In [110]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 182892 entries, 0 to 194672
Data columns (total 22 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   SEVERITYCODE    182892 non-null  int64         
 1   REPORTNO        182892 non-null  object        
 2   STATUS          182892 non-null  int64         
 3   ADDRTYPE        182892 non-null  object        
 4   SEVERITYCODE.1  182892 non-null  int64         
 5   SEVERITYDESC    182892 non-null  object        
 6   COLLISIONTYPE   182892 non-null  object        
 7   PERSONCOUNT     182892 non-null  int64         
 8   PEDCOUNT        182892 non-null  int64         
 9   PEDCYLCOUNT     182892 non-null  int64         
 10  VEHCOUNT        182892 non-null  int64         
 11  INCDTTM         182892 non-null  datetime64[ns]
 12  JUNCTIONTYPE    182892 non-null  object        
 13  SDOT_COLCODE    182892 non-null  int64         
 14  UNDERINFL       182892 non-null  int

In [111]:
# Separate majority and minority classes
df_majority = df[df['SEVERITYCODE'] == 1]
df_minority = df[df['SEVERITYCODE'] == 2]

In [112]:
# Downsample majority class
length = len(df_minority['SEVERITYCODE'])
df_majority_downsampled = resample(df_majority, 
                                 replace = False,    # sample without replacement
                                 n_samples = length,     # to match minority class
                                 random_state = 0) # reproducible results

In [115]:
# Combine minority class with downsampled majority class
df_downsampled = pd.concat([df_majority_downsampled, df_minority])